In [ ]:
#Для демо используется датасет https://www.kaggle.com/datasets/nelgiriyewithana/apple-quality/data

In [85]:
from pyspark.sql import SparkSession
from datetime import datetime, date
from pyspark.sql import Row
 
ADMIN_REFRESH_TOKEN = "<токен доступа>"
 
spark_connect_url = f"sc://10.0.5.34:15002/;spark-token={ADMIN_REFRESH_TOKEN}"
 
spark = SparkSession.builder.remote(spark_connect_url).appName("Apple_ML").getOrCreate()
 
df = spark.read.csv("s3a://spark-k8s-6721-k8s-3d150103e71aaa-bucket/datasets/apple_quality.csv", header=True, inferSchema=True)

df.show()

+----+------------+------------+------------+------------+------------+------------+------------+-------+
|A_id|        Size|      Weight|   Sweetness| Crunchiness|   Juiciness|    Ripeness|     Acidity|Quality|
+----+------------+------------+------------+------------+------------+------------+------------+-------+
|   0|-3.970048523|-2.512336381| 5.346329613|-1.012008712| 1.844900361| 0.329839797|-0.491590483|   good|
|   1|-1.195217191|-2.839256528| 3.664058758| 1.588232309| 0.853285795| 0.867530082|-0.722809367|   good|
|   2|-0.292023862|-1.351281995|-1.738429162|-0.342615928| 2.838635512|-0.038033328| 2.621636473|    bad|
|   3|-0.657195773|-2.271626609| 1.324873847|-0.097874716| 3.637970491|-3.413761338| 0.790723217|   good|
|   4|  1.36421682|-1.296611877|-0.384658206| -0.55300577| 3.030874354|-1.303849429| 0.501984036|   good|
|   5|-3.425399755|-1.409082204|-1.913511195|-0.555774864| -3.85307147| 1.914615916|-2.981523169|    bad|
|   6| 1.331605736| 1.635955715| 0.875974244| 

In [44]:
# Просмотр схемы данных
df.printSchema()

root
 |-- A_id: integer (nullable = true)
 |-- Size: double (nullable = true)
 |-- Weight: double (nullable = true)
 |-- Sweetness: double (nullable = true)
 |-- Crunchiness: double (nullable = true)
 |-- Juiciness: double (nullable = true)
 |-- Ripeness: double (nullable = true)
 |-- Acidity: string (nullable = true)
 |-- Quality: string (nullable = true)



In [43]:
df.count()

4001

In [8]:
# Общая статистика
df.describe().show()

+-------+------------------+-------------------+-------------------+--------------------+------------------+------------------+------------------+--------------------+-------+
|summary|              A_id|               Size|             Weight|           Sweetness|       Crunchiness|         Juiciness|          Ripeness|             Acidity|Quality|
+-------+------------------+-------------------+-------------------+--------------------+------------------+------------------+------------------+--------------------+-------+
|  count|              4000|               4000|               4000|                4000|              4000|              4000|              4000|                4001|   4000|
|   mean|            1999.5|-0.5030146298267509|-0.9895465445945022|-0.47047851978824984| 0.985477903858501|0.5121179684932494|0.4982774280305005| 0.07687729571600004|   NULL|
| stddev|1154.8448669265786|   1.92805868885498| 1.6025072141517547|  1.9434406589204516|1.4027572042119638|1.9302856730

In [9]:
# Статистика по конкретным колонкам
df.select("Size", "Weight", "Sweetness").describe().show()

+-------+-------------------+-------------------+--------------------+
|summary|               Size|             Weight|           Sweetness|
+-------+-------------------+-------------------+--------------------+
|  count|               4000|               4000|                4000|
|   mean|-0.5030146298267509|-0.9895465445945022|-0.47047851978824984|
| stddev|   1.92805868885498| 1.6025072141517547|  1.9434406589204516|
|    min|       -7.151703059|       -7.149847675|        -6.894485494|
|    max|        6.406366899|         5.79071359|         6.374915513|
+-------+-------------------+-------------------+--------------------+



In [13]:
from pyspark.sql import functions as F

df.groupBy("Quality").count().show()

+-------+-----+
|Quality|count|
+-------+-----+
|   NULL|    1|
|    bad| 1996|
|   good| 2004|
+-------+-----+



In [59]:
#Чистим строку с Null
df = df.filter(df.Quality.isNotNull())
df.count()

4000

In [38]:
from pyspark.sql.window import Window

# Процентное соотношение яблок по качеству
print("Процентное соотношение яблок по качеству:")
window = Window.partitionBy()  # создаем пустое окно для вычисления общей суммы
df.groupBy("Quality").count().withColumn(
    "Percentage", 
    F.round(F.col("count") / F.sum("count").over(window) * 100, 2)
).show()

Процентное соотношение яблок по качеству:


/home/datadisk/jupyter-admin/.local/lib/python3.9/site-packages/pyspark/sql/connect/expressions.py:948: UserWarning: WARN WindowExpression: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
  warnings.warn(


+-------+-----+----------+
|Quality|count|Percentage|
+-------+-----+----------+
|    bad| 1996|      49.9|
|   good| 2004|      50.1|
+-------+-----+----------+



In [23]:
# Средние значения характеристик по качеству
print("Средние значения характеристик по качеству:")

df.groupBy("Quality").agg(
    F.round(F.avg("Size"), 2).alias("Avg_Size"),
    F.round(F.avg("Weight"), 2).alias("Avg_Weight"),
    F.round(F.avg("Sweetness"), 2).alias("Avg_Sweetness"),
    F.round(F.avg("Crunchiness"), 2).alias("Avg_Crunchiness"),
    F.round(F.avg("Juiciness"), 2).alias("Avg_Juiciness")
).show()

Средние значения характеристик по качеству:
+-------+--------+----------+-------------+---------------+-------------+
|Quality|Avg_Size|Avg_Weight|Avg_Sweetness|Avg_Crunchiness|Avg_Juiciness|
+-------+--------+----------+-------------+---------------+-------------+
|    bad|   -0.97|     -0.99|        -0.96|            1.0|         0.01|
|   good|   -0.03|     -0.99|         0.02|           0.97|         1.01|
+-------+--------+----------+-------------+---------------+-------------+



In [60]:
# Анализ распределения характеристик
print("Распределение характеристик для каждого качества:")
for characteristic in ["Size", "Weight", "Sweetness", "Crunchiness", "Juiciness", "Ripeness", "Acidity"]:
    print(f"\nРаспределение {characteristic}:")
    df.groupBy("Quality").agg(
        F.round(F.min(characteristic), 2).alias(f"Min_{characteristic}"),
        F.round(F.percentile_approx(characteristic, 0.25), 2).alias(f"Q1_{characteristic}"),
        F.round(F.percentile_approx(characteristic, 0.5), 2).alias(f"Median_{characteristic}"),
        F.round(F.percentile_approx(characteristic, 0.75), 2).alias(f"Q3_{characteristic}"),
        F.round(F.max(characteristic), 2).alias(f"Max_{characteristic}")
    ).orderBy("Quality").show()

Распределение характеристик для каждого качества:

Распределение Size:
+-------+--------+-------+-----------+-------+--------+
|Quality|Min_Size|Q1_Size|Median_Size|Q3_Size|Max_Size|
+-------+--------+-------+-----------+-------+--------+
|    bad|   -7.15|  -2.18|      -1.02|   0.22|    4.65|
|   good|   -6.91|  -1.38|       0.03|   1.34|    6.41|
+-------+--------+-------+-----------+-------+--------+


Распределение Weight:
+-------+----------+---------+-------------+---------+----------+
|Quality|Min_Weight|Q1_Weight|Median_Weight|Q3_Weight|Max_Weight|
+-------+----------+---------+-------------+---------+----------+
|    bad|     -5.75|    -1.84|        -0.98|    -0.15|      3.08|
|   good|     -7.15|    -2.22|         -1.0|     0.29|      5.79|
+-------+----------+---------+-------------+---------+----------+


Распределение Sweetness:
+-------+-------------+------------+----------------+------------+-------------+
|Quality|Min_Sweetness|Q1_Sweetness|Median_Sweetness|Q3_Sweetness

In [88]:
from pyspark.sql.functions import col, when
from pyspark.sql.types import DoubleType

df = df.withColumn("Acidity", col("Acidity").cast(DoubleType()))

def detect_outliers(df, column):
    # Вычисляем квартили
    quantiles = df.approxQuantile(column, [0.25, 0.75], 0.05)
    Q1 = quantiles[0]
    Q3 = quantiles[1]
    IQR = Q3 - Q1
    
    # Определяем границы для выбросов
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    # Помечаем выбросы
    return df.withColumn(f"{column}_outlier", 
                         when((col(column) < lower_bound) | (col(column) > upper_bound), 1).otherwise(0))

# Применяем функцию ко всем числовым колонкам
numeric_columns = ["Size", "Weight", "Sweetness", "Crunchiness", "Juiciness", "Ripeness", "Acidity"]
for column in numeric_columns:
    df = detect_outliers(df, column)

# Подсчитываем количество выбросов для каждой характеристики
outlier_counts = df.select([F.sum(f"{col}_outlier").alias(f"{col}_outliers") for col in numeric_columns])
outlier_counts.show()

+-------------+---------------+------------------+--------------------+------------------+-----------------+----------------+
|Size_outliers|Weight_outliers|Sweetness_outliers|Crunchiness_outliers|Juiciness_outliers|Ripeness_outliers|Acidity_outliers|
+-------------+---------------+------------------+--------------------+------------------+-----------------+----------------+
|           56|            115|                95|                 105|                73|               71|              64|
+-------------+---------------+------------------+--------------------+------------------+-----------------+----------------+



In [89]:
from pyspark.sql.functions import when, count

def analyze_quality_relation(df, column):
    # Разделяем значения на категории
    categorized = df.withColumn(f"{column}_category",
        when(col(column) <= df.approxQuantile(column, [0.33], 0.05)[0], "Low")
        .when(col(column) <= df.approxQuantile(column, [0.66], 0.05)[0], "Medium")
        .otherwise("High")
    )
    
    # Подсчитываем распределение качества для каждой категории
    distribution = categorized.groupBy(f"{column}_category") \
        .agg(count(when(col("Quality") == "good", 1)).alias("good_count"),
             count(when(col("Quality") == "bad", 1)).alias("bad_count")) \
        .withColumn("good_percentage", col("good_count") / (col("good_count") + col("bad_count")) * 100)
    
    return distribution

# Применяем анализ ко всем числовым колонкам
for column in numeric_columns:
    print(f"\nАнализ влияния {column} на качество:")
    analyze_quality_relation(df, column).show()


Анализ влияния Size на качество:
+-------------+----------+---------+-----------------+
|Size_category|good_count|bad_count|  good_percentage|
+-------------+----------+---------+-----------------+
|         High|       959|      540|63.97598398932621|
|          Low|       513|      835|38.05637982195846|
|       Medium|       532|      621|46.14050303555941|
+-------------+----------+---------+-----------------+


Анализ влияния Weight на качество:
+---------------+----------+---------+------------------+
|Weight_category|good_count|bad_count|   good_percentage|
+---------------+----------+---------+------------------+
|           High|       797|      702|53.168779186124084|
|            Low|       728|      620| 54.00593471810089|
|         Medium|       479|      674| 41.54379878577624|
+---------------+----------+---------+------------------+


Анализ влияния Sweetness на качество:
+------------------+----------+---------+-----------------+
|Sweetness_category|good_count|bad_cou

In [10]:
spark.stop()